# KOPIS 공연추천 시스템 테스트

KOPIS OpenAPI를 활용하여 공연 정보를 수집하고, OCR과 FastText를 이용한 추천 시스템을 구현합니다.

## 필요한 라이브러리 설치

## 라이브러리 임포트

In [37]:
import cv2
import requests
import pandas as pd
import numpy as np
from PIL import Image, ImageEnhance
import pytesseract
from io import BytesIO
from gensim.models import FastText as FastText
import re
from datetime import datetime, timedelta
import xml.etree.ElementTree as ET
from typing import Optional, Dict, Any, List
from dotenv import load_dotenv
import os

## KOPIS API 클라이언트 클래스 정의

In [38]:
# .env 파일 로드
load_dotenv()

# API 키를 .env 파일에서 가져오기
KOPIS_API_KEY = os.getenv('KOPIS_API_KEY')

In [39]:
class KopisAPI:
    def __init__(self, service_key):
        self.service_key = service_key
        self.base_url = "http://www.kopis.or.kr/openApi/restful"
    
    def get_performance_list(self, start_date, end_date):
        """공연 목록 조회"""
        url = f"{self.base_url}/pblprfr"
        params = {
            'service': self.service_key,
            'stdate': start_date,
            'eddate': end_date,
            'rows': 100,
            'cpage': 1
        }
        response = requests.get(url, params=params)
        root = ET.fromstring(response.content)
        
        performances = []
        for db in root.findall('.//db'):
            perf = {}
            for child in db:
                perf[child.tag] = child.text
            performances.append(perf)
        
        return performances
    
    def get_performance_detail(self, mt20id: str) -> Optional[Dict[str, Any]]:
        """공연 상세정보 조회 - 포스터와 소개이미지 모두 처리"""
        url = f"{self.base_url}/pblprfr/{mt20id}"
        params = {'service': self.service_key}
        
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
            
            root = ET.fromstring(response.content)
            db = root.find('.//db')
            
            if db is None:
                return None
                
            detail = {}
            for elem in db:
                if elem.tag == 'styurls':
                    # XML 구조 디버깅
                    print(f"styurls element found for {mt20id}")
                    print(f"styurls content: {ET.tostring(elem, encoding='unicode')}")
                    
                    # 소개이미지 목록 추출 (수정된 XPath)
                    urls = []
                    for styurl in elem.findall('styurl'):
                        if styurl.text and styurl.text.strip():
                            print(f"Found image URL: {styurl.text}")
                            urls.append(styurl.text.strip())
                    detail['styurls'] = urls
                else:
                    if elem.text and elem.text.strip():
                        detail[elem.tag] = elem.text.strip()
                    
            # 디버깅을 위한 출력
            if 'styurls' in detail:
                print(f"Total styurls found for {mt20id}: {len(detail['styurls'])}")
            else:
                print(f"No styurls found for {mt20id}")
                
            return detail
            
        except Exception as e:
            print(f"API 요청 오류: {e}")
            return None

## 텍스트 처리 클래스 정의

In [40]:
class TextProcessor:
    def __init__(self):
        self.model = None
        pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
    
    def enhance_image(self, img):
        """이미지 품질 개선"""
        # PIL Image를 사용한 개선
        enhancer = ImageEnhance.Contrast(img)
        img = enhancer.enhance(2.0)  # 대비 증가
        enhancer = ImageEnhance.Sharpness(img)
        img = enhancer.enhance(2.0)  # 선명도 증가
        return img
    
    def preprocess_image(self, img_array):
        """OpenCV를 사용한 이미지 전처리"""
        # 그레이스케일 변환
        gray = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
        
        # 노이즈 제거
        denoised = cv2.fastNlMeansDenoising(gray)
        
        # 이진화
        _, binary = cv2.threshold(denoised, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        
        # 모폴로지 연산으로 텍스트 영역 강화
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
        processed = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
        
        return processed
    
    def get_image_sections(self, img):
        """이미지를 여러 섹션으로 분할"""
        width, height = img.size
        sections = []
        
        # 세로로 3등분
        section_height = height // 3
        for i in range(3):
            top = i * section_height
            bottom = (i + 1) * section_height
            section = img.crop((0, top, width, bottom))
            sections.append(section)
        
        return sections
    
    def extract_text_from_image(self, image_url):
        """개선된 이미지 텍스트 추출 - GIF 처리 추가"""
        try:
            print(f"이미지 다운로드 시도: {image_url}")
            response = requests.get(image_url)
            img = Image.open(BytesIO(response.content))
            
            # GIF 처리 추가
            if img.format == 'GIF':
                img = img.convert('RGB')
            
            # 이미지 크기 정규화
            target_width = 1000
            width_percent = (target_width / float(img.size[0]))
            target_height = int(float(img.size[1]) * float(width_percent))
            img = img.resize((target_width, target_height), Image.Resampling.LANCZOS)
            
            # PIL 이미지 개선
            enhanced_img = self.enhance_image(img)
            
            # OpenCV 전처리
            img_array = np.array(enhanced_img)
            processed_img = self.preprocess_image(img_array)
            
            # 이미지를 섹션으로 분할
            sections = self.get_image_sections(img)
            
            texts = []
            
            # 각 섹션별로 OCR 수행
            for i, section in enumerate(sections):
                # 다양한 OCR 설정으로 시도
                configs = [
                    '--oem 3 --psm 6',  # 기본 설정
                    '--oem 3 --psm 1',  # 자동 페이지 세그멘테이션
                    '--oem 3 --psm 4'   # 컬럼으로 가정
                ]
                
                section_texts = []
                for config in configs:
                    text = pytesseract.image_to_string(
                        section, 
                        lang='kor+eng',
                        config=config
                    )
                    if text.strip():
                        section_texts.append(text)
                
                # 가장 긴 텍스트 선택
                if section_texts:
                    longest_text = max(section_texts, key=len)
                    texts.append(longest_text)
            
            # 처리된 이미지로 한 번 더 OCR
            processed_text = pytesseract.image_to_string(
                processed_img,
                lang='kor+eng',
                config='--oem 3 --psm 6'
            )
            texts.append(processed_text)
            
            # 모든 텍스트 결합 및 정제
            combined_text = ' '.join(texts)
            cleaned_text = self.clean_text(combined_text)
            
            # 결과 로깅
            print(f"추출된 총 텍스트 길이: {len(cleaned_text)}")
            print(f"텍스트 샘플: {cleaned_text[:200]}...")
            
            return cleaned_text
            
        except Exception as e:
            print(f"이미지 처리 중 오류 발생: {str(e)}")
            return ""
    
    def clean_text(self, text):
        """텍스트 전처리"""
        if not text:
            return ""
            
        # 불필요한 문자 제거
        text = re.sub(r'[^\w\s가-힣]', ' ', text)
        
        # 연속된 공백 제거
        text = re.sub(r'\s+', ' ', text)
        
        # 줄바꿈 통일
        text = text.replace('\n', ' ')
        
        # 불필요한 반복 제거
        words = text.split()
        words = list(dict.fromkeys(words))  # 중복 제거
        text = ' '.join(words)
        
        return text.strip().lower()

    def train_model(self, texts):
        """FastText 모델 학습"""
        texts = [text for text in texts if text.strip()]
        if not texts:
            print("경고: 학습할 텍스트가 없습니다.")
            return
            
        sentences = [[word for word in text.split()] for text in texts]
        try:
            self.model = FastText(
                sentences=sentences, 
                vector_size=100, 
                window=5, 
                min_count=1,
                workers=4
            )
            print(f"모델 학습 완료: {len(sentences)} 문장")
        except Exception as e:
            print(f"모델 학습 오류: {str(e)}")
    
    def get_text_vector(self, text):
        """텍스트 벡터화"""
        if self.model is None:
            print("경고: 모델이 학습되지 않았습니다.")
            return np.zeros(100)
            
        words = text.split()
        word_vectors = [self.model.wv[word] for word in words if word in self.model.wv]
        if not word_vectors:
            return np.zeros(100)
        return np.mean(word_vectors, axis=0)

## 공연 추천 시스템 클래스 정의

In [41]:
class PerformanceRecommender:
    def __init__(self, api_client, text_processor):
        self.api_client = api_client
        self.text_processor = text_processor
        self.performances_df = None
    
    def collect_performance_data(self, days=30):
        """공연 데이터 수집 - 모든 이미지 처리"""
        start_date = datetime.now().strftime("%Y%m%d")
        end_date = (datetime.now() + timedelta(days=days)).strftime("%Y%m%d")
        
        performances = []
        perf_list = self.api_client.get_performance_list(start_date, end_date)
        
        for perf in perf_list[:10]:  # 테스트를 위해 10개만
            mt20id = perf['mt20id']
            detail = self.api_client.get_performance_detail(mt20id)
            
            if detail:
                # 포스터 텍스트 추출
                poster_text = ""
                if 'poster' in detail and detail['poster'] and detail['poster'].startswith('http'):
                    try:
                        poster_text = self.text_processor.extract_text_from_image(detail['poster'])
                    except Exception as e:
                        print(f"포스터 이미지 처리 오류({mt20id}): {str(e)}")
                
                # 소개이미지 텍스트 추출
                intro_texts = []
                if 'styurls' in detail and isinstance(detail['styurls'], list):
                    for img_url in detail['styurls']:
                        if img_url and img_url.startswith('http'):
                            try:
                                text = self.text_processor.extract_text_from_image(img_url)
                                if text:
                                    intro_texts.append(text)
                            except Exception as e:
                                print(f"소개이미지 처리 오류({mt20id}): {str(e)}")
                
                # 모든 텍스트 결합
                all_text = ' '.join(filter(None, [poster_text] + intro_texts))
                
                performances.append({
                    'mt20id': mt20id,
                    'title': detail.get('prfnm', ''),
                    'plot': all_text if all_text.strip() else ""
                })
        
        self.performances_df = pd.DataFrame(performances)
        return self.performances_df
    
    def prepare_model(self):
        """추천 모델 준비"""
        if self.performances_df is None:
            raise ValueError("공연 데이터를 먼저 수집하세요.")
        
        plots = self.performances_df['plot'].tolist()
        self.text_processor.train_model(plots)
    
    def get_recommendations(self, user_plot, top_n=5):
        """사용자 입력에 기반한 공연 추천"""
        if self.performances_df is None:
            raise ValueError("공연 데이터를 먼저 수집하세요.")
        
        user_vector = self.text_processor.get_text_vector(user_plot)
        
        # 각 공연과의 유사도 계산
        similarities = []
        for plot in self.performances_df['plot']:
            plot_vector = self.text_processor.get_text_vector(plot)
            similarity = np.dot(user_vector, plot_vector) / (
                np.linalg.norm(user_vector) * np.linalg.norm(plot_vector)
            )
            similarities.append(similarity)
        
        self.performances_df['similarity'] = similarities
        recommendations = self.performances_df.nlargest(top_n, 'similarity')
        return recommendations[['title', 'similarity']]

## 시스템 테스트

아래 셀에서 실제 테스트를 수행합니다. API 키를 설정하고 실행해보세요.

In [ ]:
# 테스트 코드
if __name__ == "__main__":
    # API 키 설정
    SERVICE_KEY = KOPIS_API_KEY
    
    # 시스템 초기화
    api_client = KopisAPI(SERVICE_KEY)
    text_processor = TextProcessor()
    recommender = PerformanceRecommender(api_client, text_processor)
    
    # 데이터 수집
    print("데이터 수집 중...")
    performances_df = recommender.collect_performance_data()
    print("\n수집된 공연 데이터:")
    print(performances_df.head())

데이터 수집 중...
styurls element found for PF256465
styurls content: <styurls>
            <styurl>http://www.kopis.or.kr/upload/pfmIntroImage/PF_PF256465_241230_0231451.jpg</styurl>
            <styurl>http://www.kopis.or.kr/upload/pfmIntroImage/PF_PF256465_241230_0231450.jpg</styurl>
        </styurls>
        
Found image URL: http://www.kopis.or.kr/upload/pfmIntroImage/PF_PF256465_241230_0231451.jpg
Found image URL: http://www.kopis.or.kr/upload/pfmIntroImage/PF_PF256465_241230_0231450.jpg
Total styurls found for PF256465: 2
이미지 다운로드 시도: http://www.kopis.or.kr/upload/pfmPoster/PF_PF256465_241230_143145.jpg
추출된 총 텍스트 길이: 347
텍스트 샘플: ws tie 0 aliquam et donec vehicula 18 p clementum est in arcu feugiat ante mere id c estibulum faucibus selerisque phasellus sed es ccu eu venenati ue luctus od ctor1ec 우구 개 로 nh donarld margulis 작 허재...
이미지 다운로드 시도: http://www.kopis.or.kr/upload/pfmIntroImage/PF_PF256465_241230_0231451.jpg
추출된 총 텍스트 길이: 347
텍스트 샘플: ws tie 0 aliquam et donec vehicula 18 p cle

In [16]:
pd.set_option('display.max_colwidth', None)

In [25]:
performances_df

,mt20id,title,plot
0,PF256465,컬렉티드 스토리즈 [인천],et donec vehicula clemenuum cottitaa e stories donarld margulis 4 허재성 연출 2025 01 15_19 토 일 4 00 p m 포동 다락소극장 으 rat rm 의 032 777 1959 qo 연문 컬렉티드 관람료 전 s 스토리즈 전석 a 000원 예매 인터파 떼아뜨르 다락 리우컴퍼니
1,PF256463,최용석 귀국 피아노 독주회,
2,PF256461,처음만난 그날 처럼 [대학로],조원오피레이터빅사 mag
3,PF256460,Seungeun Lee Quartet: KALT UND SCHWER.,free improvisation vol 2 kalt und schwer seung eun lee quartet jan artist seungeun lee piano saembawy han sitar joongwon hwang guitar 1 2 sunki kim drums 5pm visual sunki kim
4,PF256450,소중한하루,
5,PF256449,이야기 안데르센 [서울 송파],사사바 삼삼발전소가만드는 5 우는 s52 삼삼과 지혜를 키우는 동화극 kaw ssr al kkk 과천 한마당축제 겸연대회 ha 안산국제꺼리극축제 heit now 대상 jot 극 부분 무수 고양호충메술축짜도전 bylafao poe 물참고 보 세 es bal 2 10 10 1 20 se 11 20 13 30 seems 662 옥아종합지원센터 yue eantes osose n
6,PF256447,"기획초청 Pick크닉, 유원",2025기티초휘003니 제작 앤드씨어터 oo 유원
7,PF256442,오백에 삼십 [대구],교 므 fu 8 브 o 수 do ofr se fr fy se bn cf so safes 40 od sy 2 ay oh od 48 4 에 지벌이다 age ng 30일에 ews 및 공휴일 24 a 20503 2025 0427 ap btn 308 bnl ons 아트플거스찌어더 문의 010 7151 7679
8,PF256440,Beethoven Relay: 플루티스트 조성현 & 앙상블,
9,PF256439,"제1회 123 페스티벌, 하녀들",sey we 2025 1 1 1 12 2 5 29 2025 1 14 1 28 49000009패 이


In [26]:
# 모델 학습
print("모델 학습 중...")
recommender.prepare_model()

모델 학습 중...
모델 학습 완료: 7 문장


In [27]:
# 추천 테스트
test_input = "로맨틱한 분위기의 뮤지컬"
print(f"테스트 입력: {test_input}")

recommendations = recommender.get_recommendations(test_input)
print("\n추천 결과:")
display(recommendations)

테스트 입력: 로맨틱한 분위기의 뮤지컬

추천 결과:


C:\Users\USER\AppData\Local\Temp\ipykernel_4900\1396570268.py:49: RuntimeWarning: invalid value encountered in scalar divide
  similarity = np.dot(user_vector, plot_vector) / (


,title,similarity
0,컬렉티드 스토리즈 [인천],0.154465
5,이야기 안데르센 [서울 송파],0.032619
2,처음만난 그날 처럼 [대학로],0.010577
9,"제1회 123 페스티벌, 하녀들",-0.021760
7,오백에 삼십 [대구],-0.025576


## 결과 분석

위 결과를 바탕으로 다음과 같은 분석이 가능합니다:
1. 추천된 공연들의 유사도 점수 분포
2. OCR 텍스트 추출 품질
3. FastText 모델의 성능

필요한 경우 아래 셀에서 추가 분석을 수행할 수 있습니다.